In [1]:
import re
ex = """on x=10..12,y=10..12,z=10..12
on x=11..13,y=11..13,z=11..13
off x=9..11,y=9..11,z=9..11
on x=10..10,y=10..10,z=10..10""".splitlines()

def parse(inp):
    res = []
    for l in inp:
        state, _ = l.split(" ", 2)
        res.append({
            "action": state == "on",
            "cords": list(map(int, re.findall(r'-?\d+', l)))
        })
    return res

parse(ex)

[{'action': True, 'cords': [10, 12, 10, 12, 10, 12]},
 {'action': True, 'cords': [11, 13, 11, 13, 11, 13]},
 {'action': False, 'cords': [9, 11, 9, 11, 9, 11]},
 {'action': True, 'cords': [10, 10, 10, 10, 10, 10]}]

In [2]:
from collections import namedtuple, defaultdict
from itertools import product
P = namedtuple("P", ["x", "y", "z"])

def in_bounds(p: P):
    return (
        (-50 <= p.x <= 50) and
        (-50 <= p.y <= 50) and
        (-50 <= p.z <= 50)
    )

def to_bounds(n):
    if n>50:
        return 50
    if n<-50:
        return -50
    return n

def get_range(cords, i):
    low = min(cords[i], cords[i+1]+1)
    high = max(cords[i], cords[i+1]+1)
    low = to_bounds(low)
    high = to_bounds(high)
    return range(low, high)

def part1(inp):
    steps = parse(inp)
    g: dict[P, int] = defaultdict(bool)
    for s in steps:
        for x,y,z in product(
            get_range(s["cords"], 0),
            get_range(s["cords"], 2),
            get_range(s["cords"], 4)
        ):
            p = P(x,y,z)
            if in_bounds(p):
                g[P(x,y,z)] = s["action"]
    return sum(v for p,v in g.items() if in_bounds(p))
part1(ex)


39

In [3]:
ex2 = """on x=-20..26,y=-36..17,z=-47..7
on x=-20..33,y=-21..23,z=-26..28
on x=-22..28,y=-29..23,z=-38..16
on x=-46..7,y=-6..46,z=-50..-1
on x=-49..1,y=-3..46,z=-24..28
on x=2..47,y=-22..22,z=-23..27
on x=-27..23,y=-28..26,z=-21..29
on x=-39..5,y=-6..47,z=-3..44
on x=-30..21,y=-8..43,z=-13..34
on x=-22..26,y=-27..20,z=-29..19
off x=-48..-32,y=26..41,z=-47..-37
on x=-12..35,y=6..50,z=-50..-2
off x=-48..-32,y=-32..-16,z=-15..-5
on x=-18..26,y=-33..15,z=-7..46
off x=-40..-22,y=-38..-28,z=23..41
on x=-16..35,y=-41..10,z=-47..6
off x=-32..-23,y=11..30,z=-14..3
on x=-49..-5,y=-3..45,z=-29..18
off x=18..30,y=-20..-8,z=-3..13
on x=-41..9,y=-7..43,z=-33..15
on x=-54112..-39298,y=-85059..-49293,z=-27449..7877
on x=967..23432,y=45373..81175,z=27513..53682""".splitlines()

part1(ex2)

588432

In [4]:
with open("data/22.txt") as f:
    pzl = f.readlines()

part1(pzl)

583641

# Part 2


In [39]:
import re
ex = """on x=10..12,y=10..12,z=10..12
on x=11..13,y=11..13,z=11..13
off x=9..11,y=9..11,z=9..11
on x=10..10,y=10..10,z=10..10""".splitlines()

def parse(inp):
    res = []
    for l in inp:
        state, _ = l.split(" ", 2)
        res.append({
            "action": state == "on",
            "cords": list(map(int, re.findall(r'-?\d+', l)))
        })
    return res

seq = parse(ex)
seq

[{'action': True, 'cords': [10, 12, 10, 12, 10, 12]},
 {'action': True, 'cords': [11, 13, 11, 13, 11, 13]},
 {'action': False, 'cords': [9, 11, 9, 11, 9, 11]},
 {'action': True, 'cords': [10, 10, 10, 10, 10, 10]}]

In [40]:
from collections import namedtuple
from itertools import product
P = namedtuple("P", ["x", "y", "z"])

class Cube():
    def __init__(self, cords: list[int]) -> None:
        x1,x2,y1,y2,z1,z2 = cords
        self.corners = tuple(P(x,y,z) for x,y,z in product([x1, x2], [y1, y2], [z1, z2]))
        self.x_low = min(x1, x2)
        self.x_high = max(x1, x2)
        self.y_low = min(y1, y2)
        self.y_high = max(y1, y2)
        self.z_low = min(z1, z2)
        self.z_high = max(z1, z2)
    def __repr__(self) -> str:
        return ",".join(map(str, [self.x_low, self.x_high, self.y_low, self.y_high, self.z_low, self.z_high]))

def in_cube(p: P, c: Cube) -> bool:
    return (
        c.x_low <= p.x <= c.x_high and 
        c.y_low <= p.y <= c.y_high and 
        c.z_low <= p.z <= c.z_high
    )
def intersect(c1, c2: Cube) -> bool:
    return (
        c1.x_high >= c2.x_low and
        c1.x_low <= c2.x_high and
        c1.y_high >= c2.y_low and
        c1.y_low <= c2.y_high and
        c1.z_high >= c2.z_low and
        c1.z_low <= c2.z_high
    )

def inside(c1: Cube, c2: Cube):
    return (
        c1.x_low >= c2.x_low and
        c1.x_high <= c2.x_high and
        c1.y_low >= c2.y_low and
        c1.y_high <= c2.y_high and
        c1.z_low >= c2.z_low and
        c1.z_high <= c2.z_high 
    )

cubes = []
for s in seq:
    print(s)
    c = Cube(s["cords"])
    if s["action"]:
        cubes.append(c)
        continue

    for other in cubes[:]:
        # nothing in that cube is being turned off
        if not intersect(c, other):
            continue

        # the cube is entirely turned off
        if inside(other, c):
            cubes.remove(other)
            continue
        print("intersectin!", c, other)

cubes

{'action': True, 'cords': [10, 12, 10, 12, 10, 12]}
{'action': True, 'cords': [11, 13, 11, 13, 11, 13]}
{'action': False, 'cords': [9, 11, 9, 11, 9, 11]}
intersectin! 9,11,9,11,9,11 10,12,10,12,10,12
intersectin! 9,11,9,11,9,11 11,13,11,13,11,13
{'action': True, 'cords': [10, 10, 10, 10, 10, 10]}


[10,12,10,12,10,12, 11,13,11,13,11,13, 10,10,10,10,10,10]

In [31]:

print(inside(cubes[2], cubes[0]))

def partially_off(off: Cube, on: Cube):
    pass
    


True
